<a href="https://colab.research.google.com/github/vilsonrodrigues/PegasusProject/blob/master/Pegasus_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando bibliotecas

In [0]:
import pandas as pd
import numpy as np

In [5]:
tab = pd.read_csv("https://raw.githubusercontent.com/vilsonrodrigues/PegasusProject/master/Data%20Bases/submissoesDiferetesEIguaisA100.csv")
tab = tab.drop(columns={"Unnamed: 0"})
tab.head(1)

,idAluno,nota,situacao,submissões diferentes e iguais a 100% semana 1,submissões diferentes e iguais a 100% semana 2,submissões diferentes e iguais a 100% semana 3,submissões diferentes e iguais a 100% semana 4,submissões diferentes e iguais a 100% semana 5,submissões diferentes e iguais a 100% semana 6,submissões diferentes e iguais a 100% semana 7,submissões diferentes e iguais a 100% semana 8,submissões diferentes e iguais a 100% semana 9,submissões diferentes e iguais a 100% semana 10,submissões diferentes e iguais a 100% semana 11,submissões diferentes e iguais a 100% semana 12,submissões diferentes e iguais a 100% semana 13,submissões diferentes e iguais a 100% semana 14,submissões diferentes e iguais a 100% semana 15,submissões diferentes e iguais a 100% semana 16,submissões diferentes e iguais a 100% semana 17,submissões diferentes e iguais a 100% semana 18
0,5991cb910be9d42f2f68b906,7.9,APROVADO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Função pra gerar o campo dos vencedores

In [0]:
def criaColunaVencedores(table):
  for i in range(1,19):
    table["Vencedor semana "+str(i)] = 0
  return table

In [0]:
vencedorX = pd.DataFrame()
vencedorY = pd.DataFrame()

Chamada da função de criação de coluna dos vencedores

In [8]:
vencedorX = criaColunaVencedores(vencedorX)
vencedorY = criaColunaVencedores(vencedorY)
vencedorX.head(0)

,Vencedor semana 1,Vencedor semana 2,Vencedor semana 3,Vencedor semana 4,Vencedor semana 5,Vencedor semana 6,Vencedor semana 7,Vencedor semana 8,Vencedor semana 9,Vencedor semana 10,Vencedor semana 11,Vencedor semana 12,Vencedor semana 13,Vencedor semana 14,Vencedor semana 15,Vencedor semana 16,Vencedor semana 17,Vencedor semana 18


Instalando a SOM

In [0]:
!pip install minisom

Selecionando o df com o campo nota não nulo

In [0]:
tab = tab[tab["nota"] != '-']

Convertendo notas para float

In [0]:
tab["nota"] = tab["nota"].astype(float)

Separando o X treino

In [0]:
X_train = tab.iloc[:, 3:].values

Escolhendo o alvo, que será a nota final

In [13]:
nota = tab.iloc[:,1].values
nota

array([7.9, 7.9, 7.9, ..., 0. , 0. , 0. ])

Criando o modelo da SOM

In [0]:
tamanhoXdaRede = 14; 
tamanhoYdaRede = 14; 
quantidadeCaracteristicas = 18

from minisom import MiniSom
som = MiniSom(x = tamanhoXdaRede, y = tamanhoYdaRede, input_len = quantidadeCaracteristicas, sigma = 1.0, learning_rate = 0.4)

Treinando a rede SOM

In [0]:
som.train_random(data = X_train, num_iteration = 100000)

Obtem o vetor de pesos da rede treinada 

In [0]:
pesos = som.get_weights()

Obtendo os valores das repostas 

In [17]:
Y_train = nota
Y_train[1]

7.9

In [19]:
X_train[1,:]

array([ 0.,  0.,  0.,  5., 13.,  5.,  0., 15.,  9.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

Criando 3 matrizes para adicionar os vencedores 

In [0]:
# encontra o vencedor 
x = X_train[1,:]
pos = som.winner(x)

# matriz de zeros para contador de reprovados 
MContRe = np.zeros((tamanhoXdaRede,tamanhoYdaRede))
# matriz de zeros para contador de aprovados 
MContAp = np.zeros((tamanhoXdaRede,tamanhoYdaRede))
# matriz de zeros para o número total de alunos
MContT = np.zeros((tamanhoXdaRede,tamanhoYdaRede))

cont = 0; 

for x in X_train: 
  pos = som.winner(x)
  if (Y_train[cont] < 5): #Reprovado 
    MContRe[pos] += 1
  if (Y_train[cont] >= 5): #Aprovado 
    MContAp[pos] += 1    
  MContT[pos] += 1
  cont= cont+1

In [0]:
X_train_selec = table...

cont = 0
vencedorX = pd.DataFrame()
vencedorY = pd.DataFrame()

for i in X_train_selec:
  novoX = pd.DataFrame()
  novoY = pd.DataFrame()
  pos = som.winner(i)
  arraybi = np.asarray(pos)
  coordX = arraybi[:,0]
  coordY = arraybi[:,1]
  novoX["Vencedor semana "+str(i)] = coordX
  novoY["Vencedor semana "+str(i)] = coordY
  vencedorX.concat(novoX)
  vencedorY.concat(novoY)

Escolhendo um caso específico para verificar o desempenho

In [0]:
aluno = tabfinal.loc[tabfinal["idAluno"]=="5991cd870be9d42f2f68b90c",:]
aluno = aluno.iloc[:,44:62].values

BomDesempenho = np.zeros((tamanhoXdaRede,tamanhoYdaRede))

cont = 0

for x in range(tamanhoXdaRede): 
  pos = som.winner(x)    
  BomDesempenho[pos] += 1

Neste trecho este código cria uma tupla com as posições do vencedores e depois adiciona a um ndarray, que vai separar o trecho em vencedores de x e y

In [0]:
t = ()
for x in range(tamanhoXdaRede): 
  pos = som.winner(x)    
  t = t + (pos,)

arraybi = np.asarray(t)
vencedoresX = arraybi[:,0]
vencedoresY = arraybi[:,1]

In [0]:
def selecionaVencedorEInsereEmSemanas(vencedorX,vencedorY):
  t = ()
  for x in range(tamanhoXdaRede):
    pos = som.winner(x)
    MContT[pos] += 1